In [1]:
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.stats import ks_2samp
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
demographic_features = ['PRACTICE_PATIENT_ID', 'SEX', 'AGE']
interest_feature_list = ['EGFR', 'SYSTOLIC_BLOOD_PRESSURE', 'HBA1C_-_DIABETIC_CONTROL', 
                         'SERUM_CHOLESTEROL', 'HIGH_DENSITY_LIPOPROTEIN', 'LOW_DENSITY_LIPOPROTEIN', 
                         'TRIGLYCERIDES', 'BMI']
treatment_list = ['INSULIN_GOLD_OPTIMAL', 'THIAZOLIDINEDIONE_GOLD_OPTIMAL', 
                  'SULPHONYLUREAS_GOLD_OPTIMAL', 'MEGLITINIDES_GOLD_OPTIMAL', 
                  'METFORMIN_GOLD_OPTIMAL', 'GLP1_GOLD_OPTIMAL', 'ACARBOSE_GOLD_OPTIMAL', 'DPP4I', 'SGLT2']

extra_features = ['ACTION', 'REWARD_BP', 'REWARD_A1C', 'REWARD']

all_features = demographic_features+interest_feature_list+treatment_list

In [3]:
def get_patient_id(exclude_cvd: bool = True, exclude_ckd: bool = False):
    data = pd.read_csv("../data/raw/RL_Adi_T2DM_HBA1C_V220220505100211.csv")
    patients = None
    if exclude_cvd:
        patients = data[data["B.M.ALLCVD:1"] == 0]["PRACTICE_PATIENT_ID"].unique()
        
    return patients
    
patient_ids = get_patient_id()    

In [ ]:
def transform_data(value):
    if value > 0 and value < 250.0:
        value = math.floor(value*10)/10
        if value < 22:
            hbalc = 10.929 * (value - 2.15)
            return round(hbalc, 2)
        else:
            return value
    else:
        return np.clip(value, 0.0, 250.0)

    
def generate_formatted_hbalc(unit: str = "mmol/mol"):
    data = pd.read_csv("../data/AVF2_RL_Adi_T2DM_HBA1C_V220220505102401_HbA1c_-_diabetic_control_2.csv")
    data = data[data['Value'].notna()]
    data['Value'] = data['Value'].apply(transform_data)
    new_df = data.groupby(["PRACTICE_PATIENT_ID", "EVENT_DATE)"])[['Value']].mean().reset_index()
    new_df.columns = ['PRACTICE_PATIENT_ID', "EVENT_DATE", 'HBA1C_VALUE']
    return new_df
transformed_hba1c = generate_formatted_hbalc()

In [ ]:
def insert_agg_value(a1c_agg, creatinine_agg, df, visit_idx):
    if len(a1c_agg) == 0:
        df.iloc[visit_idx, df.columns.get_loc("HBA1C_-_DIABETIC_CONTROL")] = None
    else:
        df.iloc[visit_idx, df.columns.get_loc("HBA1C_-_DIABETIC_CONTROL")] = np.mean(a1c_agg)
    
    if len(creatinine_agg) == 0:
        df.iloc[visit_idx, df.columns.get_loc("SERUM_CREATININE")] = None
    else:
        df.iloc[visit_idx, df.columns.get_loc("SERUM_CREATININE")] = np.mean(creatinine_agg)

def merge_hba1c_serem(df_a1c, pat_id):
    ts_df = pd.read_csv("../data/AVF1_RL_Adi_T2DM_HBA1C_V220220505102355_TimeSeries.csv")
    df_creatinine = pd.read_csv("../data/AVF2_RL_Adi_T2DM_HBA1C_V220220505102401_Serum_creatinine_3.csv")
    ts_df = ts_df[ts_df['PRACTICE_PATIENT_ID'].isin(pat_id)]
    
    # change event date data type.
    df_creatinine['EVENT_DATE'] = df_creatinine['EVENT_DATE'].apply(lambda x: datetime.strptime(x, "%d/%m/%Y") )
    df_a1c['EVENT_DATE'] = df_a1c['EVENT_DATE'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d") )
    
    for patient, d in tqdm(ts_df.groupby(['PRACTICE_PATIENT_ID'])):
        total_num_visits = d.shape[0]
        date_start_threshold = None
        alc_data = df_a1c[df_a1c["PRACTICE_PATIENT_ID"].isin([patient])]
        creat_data = df_creatinine[df_creatinine["PRACTICE_PATIENT_ID"].isin([patient])]
        for visit_idx in range(d.shape[0]):
            if visit_idx == 0:
                date_start_threshold = datetime.strptime(d.iloc[visit_idx, d.columns.get_loc("INDEX_DATE")], "%Y-%m-%d") 
                date_end_threshold = date_start_threshold + relativedelta(months=2)
                a1c_agg = alc_data[alc_data["EVENT_DATE"] <= date_end_threshold]["HBA1C_VALUE"].values
                creatinine_agg = creat_data[creat_data["EVENT_DATE"] <= date_end_threshold]["Value"].values
                insert_agg_value(a1c_agg, creatinine_agg, d, visit_idx)
                date_start_threshold = date_end_threshold
            else:
                date_end_threshold = date_start_threshold + relativedelta(months=2)
                a1c_agg = alc_data[(alc_data["EVENT_DATE"] > date_start_threshold) & 
                                   (alc_data["EVENT_DATE"] <= date_end_threshold)]["HBA1C_VALUE"].values
                creatinine_agg = creat_data[(creat_data["EVENT_DATE"] > date_start_threshold) & 
                                            (creat_data["EVENT_DATE"] <= date_end_threshold)]["Value"].values
                insert_agg_value(a1c_agg, creatinine_agg, d, visit_idx)
                date_start_threshold = date_end_threshold
    return ts_df

ts_df = merge_hba1c_serem(transformed_hba1c, patient_ids)

In [ ]:
ts_df.to_csv("baseline.csv", index=False)

In [4]:
ts_df = pd.read_csv("baseline.csv")

In [5]:
ftr_list = ['SYSTOLIC_BLOOD_PRESSURE', 'BMI', 'HIGH_DENSITY_LIPOPROTEIN', 'LOW_DENSITY_LIPOPROTEIN', 'SERUM_CHOLESTEROL', 
 'HBA1C_-_DIABETIC_CONTROL', 'TRIGLYCERIDES', 'SERUM_CREATININE']

for ftr in ftr_list:
    ts_df[ftr] = ts_df.groupby('PRACTICE_PATIENT_ID')[ftr].transform(lambda v: v.ffill())
    ts_df[ftr] = ts_df.groupby('PRACTICE_PATIENT_ID')[ftr].transform(lambda v: v.bfill())

In [6]:
def calculate_egfr(serumCreatinineValue, age, sex, ethnicity, a, k):
    CKD_STAGE = ""
    min_val = math.pow(min((serumCreatinineValue / 88.42) / k, 1.0)+1e-8, a)
    max_val = math.pow(max((serumCreatinineValue / 88.42) / k, 1.0), -1.209)
    gfr = (141 * min_val * max_val) * (math.pow(0.993, age))
    if sex == "F": 
        gfr = gfr * 1.018
    
    if ethnicity == "BLACK":
        gfr = gfr * 1.159
    
    finale = round(gfr, 2)
    if finale < 31:
        CKD_STAGE = "STAGE-4-5"
    elif finale >= 31 and finale < 61:
        CKD_STAGE = "STAGE-3";
    elif finale >= 61:
        CKD_STAGE = "STAGE-1-2"
    else:
        CKD_STAGE = ""
    
    return finale, CKD_STAGE

def generate_egfr(data):
    k = 0.7 if data['SEX'] == 'F' else 0.9
    a = -0.329 if data['SEX'] == 'F' else -0.411
    egfr, _ = calculate_egfr(data['SERUM_CREATININE'], data['AGE'], data['SEX'], data['ETHNICITY'], a, k)
    return egfr

ts_df['EGFR'] = ts_df[['SEX', 'ETHNICITY', 'AGE', 'SERUM_CREATININE']].apply(generate_egfr, axis=1)

In [7]:
def generate_ckd(data):
    k = 0.7 if data['SEX'] == 'F' else 0.9
    a = -0.329 if data['SEX'] == 'F' else -0.411
    _, ckd = calculate_egfr(data['SERUM_CREATININE'], data['AGE'], data['SEX'], data['ETHNICITY'], a, k)
    return ckd

ts_df['CKD_STAGE'] = ts_df[['SEX', 'ETHNICITY', 'AGE', 'SERUM_CREATININE']].apply(generate_ckd, axis=1)

In [8]:
ts_df.to_csv("baseline.csv", index=False)

In [4]:
ts_df = pd.read_csv("baseline.csv")

In [5]:
ftr_list = ['SYSTOLIC_BLOOD_PRESSURE', 'BMI', 'HIGH_DENSITY_LIPOPROTEIN', 'LOW_DENSITY_LIPOPROTEIN', 'SERUM_CHOLESTEROL', 
 'HBA1C_-_DIABETIC_CONTROL', 'TRIGLYCERIDES', 'SERUM_CREATININE']
invalid_pats = []
for patient, d in tqdm(ts_df.groupby(['PRACTICE_PATIENT_ID'])):
    for ftr in ftr_list:
        if np.isnan(d.iloc[0, d.columns.get_loc(ftr)]):
            invalid_pats.append(patient)

100%|████████████████████████████████████████████████████████████████████████| 319298/319298 [01:53<00:00, 2802.86it/s]


In [6]:
ts_df = ts_df.drop(ts_df[ts_df.PRACTICE_PATIENT_ID.isin(invalid_pats)].index)

In [7]:
for ftr in ftr_list:
    print(ftr, (ts_df[ftr].isna().sum() / len(ts_df[ftr])) * 100 )

SYSTOLIC_BLOOD_PRESSURE 0.0
BMI 0.0
HIGH_DENSITY_LIPOPROTEIN 0.0
LOW_DENSITY_LIPOPROTEIN 0.0
SERUM_CHOLESTEROL 0.0
HBA1C_-_DIABETIC_CONTROL 0.0
TRIGLYCERIDES 0.0
SERUM_CREATININE 0.0


In [8]:
ts_df.columns

Index(['PRACTICE_PATIENT_ID', 'YEAR_START', 'YEAR_END', 'STATUS', 'SEX',
       'ETHNICITY', 'INTERVAL_OF_FOLLOWUP', 'DATE_OF_BIRTH', 'DEATH_DATE',
       'INDEX_DATE', 'EXIT_DATE', 'AGE', 'HYPERTENSION_BHAM_CAM',
       'HYPOTHYROIDISM_MM', 'ATYPICALANTIPSYCHOTICS', 'AF_BHAM_CAM',
       'RHEUM_ARTH_MM2', 'SYSTEMIC_LUPUS_ERYTHEMATOSUS_MM_BIRM_CAM',
       'SIGHTTHREATENINGRETINOPATHYR2R3M1', 'ASTHMALONGLIST2018',
       'DIABETES_FOOTULCER_AMPUTATION', 'SMIRR', 'ANXIETY_BIRM_CAM',
       'DEPRESSION_BIRM_CAM', 'OSAFINAL', 'ALL_DIURETICS_D2T',
       'ACE_INHIBITORS_D2T', 'MIGRAINEPREVALENT_BIRM_CAM',
       'INSULIN_GOLD_OPTIMAL', 'THIAZOLIDINEDIONE_GOLD_OPTIMAL',
       'BETABLOCKERS_OPTIMAL', 'STATIN_GOLD_OPTIMAL',
       'SULPHONYLUREAS_GOLD_OPTIMAL', 'MEGLITINIDES_GOLD_OPTIMAL',
       'CARBAMAZEPINE_OPTIMAL', 'METFORMIN_GOLD_OPTIMAL', 'SSRIS_OPTIMAL',
       'DPP4INHIBITORS_GOLD_OPTIMAL', 'DPP4INHIBITORS_OPTIMAL',
       'GLP1_GOLD_OPTIMAL', 'ACARBOSE_GOLD_OPTIMAL',
       'SGLT2

In [9]:
ts_df['DPP4I'] = ts_df[['DPP4INHIBITORS_GOLD_OPTIMAL', 'DPP4INHIBITORS_OPTIMAL']].max(axis=1)

In [10]:
ts_df['SGLT2'] = ts_df[['SGLT2INHIBITORS_GOLD_OPTIMAL', 'SGLT2INHIBITORS_OPTIMAL']].max(axis=1)

In [11]:
ts_df = ts_df[ts_df['PRACTICE_PATIENT_ID'].isin(patient_ids)]

In [12]:
ts_df.columns

Index(['PRACTICE_PATIENT_ID', 'YEAR_START', 'YEAR_END', 'STATUS', 'SEX',
       'ETHNICITY', 'INTERVAL_OF_FOLLOWUP', 'DATE_OF_BIRTH', 'DEATH_DATE',
       'INDEX_DATE', 'EXIT_DATE', 'AGE', 'HYPERTENSION_BHAM_CAM',
       'HYPOTHYROIDISM_MM', 'ATYPICALANTIPSYCHOTICS', 'AF_BHAM_CAM',
       'RHEUM_ARTH_MM2', 'SYSTEMIC_LUPUS_ERYTHEMATOSUS_MM_BIRM_CAM',
       'SIGHTTHREATENINGRETINOPATHYR2R3M1', 'ASTHMALONGLIST2018',
       'DIABETES_FOOTULCER_AMPUTATION', 'SMIRR', 'ANXIETY_BIRM_CAM',
       'DEPRESSION_BIRM_CAM', 'OSAFINAL', 'ALL_DIURETICS_D2T',
       'ACE_INHIBITORS_D2T', 'MIGRAINEPREVALENT_BIRM_CAM',
       'INSULIN_GOLD_OPTIMAL', 'THIAZOLIDINEDIONE_GOLD_OPTIMAL',
       'BETABLOCKERS_OPTIMAL', 'STATIN_GOLD_OPTIMAL',
       'SULPHONYLUREAS_GOLD_OPTIMAL', 'MEGLITINIDES_GOLD_OPTIMAL',
       'CARBAMAZEPINE_OPTIMAL', 'METFORMIN_GOLD_OPTIMAL', 'SSRIS_OPTIMAL',
       'DPP4INHIBITORS_GOLD_OPTIMAL', 'DPP4INHIBITORS_OPTIMAL',
       'GLP1_GOLD_OPTIMAL', 'ACARBOSE_GOLD_OPTIMAL',
       'SGLT2

In [13]:
invalid_id = []

for patient, d in tqdm(ts_df.groupby(['PRACTICE_PATIENT_ID'])):
    total_num_visits = d.shape[0]
    if np.any(d['EGFR'].values < 60.0):
        invalid_id.append(patient)
ts_df = ts_df.drop(ts_df[ts_df.PRACTICE_PATIENT_ID.isin(invalid_id)].index)

100%|████████████████████████████████████████████████████████████████████████| 164564/164564 [00:20<00:00, 7911.30it/s]


In [14]:
len(ts_df['PRACTICE_PATIENT_ID'].unique())

118280

In [15]:
def foo(group):
    for i in range(1,len(group)):
        if sum(group.loc[group.index[i], treatment_list].values) == 0:
            group.iloc[i,group.columns.get_loc('INSULIN_GOLD_OPTIMAL')]= group.iloc[i-1,group.columns.get_loc('INSULIN_GOLD_OPTIMAL')]
            group.iloc[i,group.columns.get_loc('THIAZOLIDINEDIONE_GOLD_OPTIMAL')]= group.iloc[i-1,group.columns.get_loc('THIAZOLIDINEDIONE_GOLD_OPTIMAL')]
            group.iloc[i,group.columns.get_loc('SULPHONYLUREAS_GOLD_OPTIMAL')]= group.iloc[i-1,group.columns.get_loc('SULPHONYLUREAS_GOLD_OPTIMAL')]
            group.iloc[i,group.columns.get_loc('MEGLITINIDES_GOLD_OPTIMAL')]= group.iloc[i-1,group.columns.get_loc('MEGLITINIDES_GOLD_OPTIMAL')]
            group.iloc[i,group.columns.get_loc('METFORMIN_GOLD_OPTIMAL')]= group.iloc[i-1,group.columns.get_loc('METFORMIN_GOLD_OPTIMAL')]
            group.iloc[i,group.columns.get_loc('GLP1_GOLD_OPTIMAL')]= group.iloc[i-1,group.columns.get_loc('GLP1_GOLD_OPTIMAL')]
            group.iloc[i,group.columns.get_loc('ACARBOSE_GOLD_OPTIMAL')]= group.iloc[i-1,group.columns.get_loc('ACARBOSE_GOLD_OPTIMAL')]
            group.iloc[i,group.columns.get_loc('DPP4I')]= group.iloc[i-1,group.columns.get_loc('DPP4I')]
            group.iloc[i,group.columns.get_loc('SGLT2')]= group.iloc[i-1,group.columns.get_loc('SGLT2')]
    return group
ts_df = ts_df.groupby('PRACTICE_PATIENT_ID').apply(foo)

In [16]:
ts_df.to_csv("clean_baseline.csv", index=False)

In [ ]:
ts_df = pd.read_csv("clean_baseline.csv")

In [17]:
subset = ts_df[(ts_df['INTERVAL_OF_FOLLOWUP'] >= 4) & (ts_df['INTERVAL_OF_FOLLOWUP'] <= 60)  & (ts_df['AGE'] >= 25) & (ts_df['AGE'] <= 80)].copy(deep=True)

In [18]:
len(subset['PRACTICE_PATIENT_ID'].unique())

98425

In [19]:
def get_action(row):
    index_l = np.where(row == 1)
    combi_drugs = ""
    for idx in index_l[0]:
        combi_drugs += treatment_list[idx].split("_")[0] + "+"
    if combi_drugs == "":
        return "LIFESTYLE_INTERVENTION"
    else:
        return combi_drugs[:-1]


subset['ACTION_DESC'] = subset.apply(lambda row: get_action(row[treatment_list]), axis=1)

In [20]:
subset["N_DRUGS"] = subset["ACTION_DESC"].apply(lambda x: len(x.split("+")))

In [21]:
subset['ACTION'] = pd.factorize(subset['ACTION_DESC'])[0]

In [25]:
subset[subset['ACTION_DESC'] == 'LIFESTYLE_INTERVENTION'].head(5)

,PRACTICE_PATIENT_ID,YEAR_START,YEAR_END,STATUS,SEX,ETHNICITY,INTERVAL_OF_FOLLOWUP,DATE_OF_BIRTH,DEATH_DATE,INDEX_DATE,...,SMOKING,TRIGLYCERIDES,SERUM_CREATININE,EGFR,CKD_STAGE,DPP4I,SGLT2,ACTION_DESC,N_DRUGS,ACTION
765,p10596_192810596,0,1,0,M,MISSING,19,1951-12-01,NaN,2010-01-01,...,NaN,1.12,86.0,85.36,STAGE-1-2,0,0,LIFESTYLE_INTERVENTION,1,4
766,p10596_192810596,1,2,0,M,MISSING,19,1951-12-01,NaN,2010-01-01,...,NaN,1.12,86.0,85.26,STAGE-1-2,0,0,LIFESTYLE_INTERVENTION,1,4
767,p10596_192810596,2,3,0,M,MISSING,19,1951-12-01,NaN,2010-01-01,...,3.0,1.12,86.0,85.17,STAGE-1-2,0,0,LIFESTYLE_INTERVENTION,1,4
768,p10596_192810596,3,4,0,M,MISSING,19,1951-12-01,NaN,2010-01-01,...,NaN,1.12,86.0,85.07,STAGE-1-2,0,0,LIFESTYLE_INTERVENTION,1,4
769,p10596_192810596,4,5,0,M,MISSING,19,1951-12-01,NaN,2010-01-01,...,NaN,1.12,86.0,84.96,STAGE-1-2,0,0,LIFESTYLE_INTERVENTION,1,4


In [26]:
subset["MEDICATION"] = subset.groupby('PRACTICE_PATIENT_ID').ACTION.shift(1,fill_value=4)

In [27]:
subset = subset[all_features+['ACTION_DESC', 'ACTION', 'MEDICATION']]

In [28]:
subset = subset.rename(columns={'HBA1C_-_DIABETIC_CONTROL': 'HBA1C',
                   'INSULIN_GOLD_OPTIMAL': 'INSULIN', 
                   'THIAZOLIDINEDIONE_GOLD_OPTIMAL': 'THIAZOLIDINEDIONE',
                   'BETABLOCKERS_OPTIMAL': 'BETABLOCKERS', 
                   'SULPHONYLUREAS_GOLD_OPTIMAL': 'SULPHONYLUREAS',
                   'MEGLITINIDES_GOLD_OPTIMAL': 'MEGLITINIDES', 
                   'METFORMIN_GOLD_OPTIMAL': 'METFORMIN',
                   'GLP1_GOLD_OPTIMAL': 'GLP1', 
                   'ACARBOSE_GOLD_OPTIMAL': 'ACARBOSE'})

In [30]:
subset["EGFR"] = subset.EGFR.clip(0.0, 90.0)
subset["SYSTOLIC_BLOOD_PRESSURE"] = subset.SYSTOLIC_BLOOD_PRESSURE.clip(0.0, 240.0)
subset["HBA1C"] = subset.HBA1C.clip(0.0, 300.0)
subset["SERUM_CHOLESTEROL"] = subset.SERUM_CHOLESTEROL.clip(0.0, 20.0)
subset["HIGH_DENSITY_LIPOPROTEIN"] = subset.HIGH_DENSITY_LIPOPROTEIN.clip(0.0, 30.0)
subset["LOW_DENSITY_LIPOPROTEIN"] = subset.LOW_DENSITY_LIPOPROTEIN.clip(0.0, 30.0)
subset["TRIGLYCERIDES"] = subset.TRIGLYCERIDES.clip(0.0, 50.0)
subset["BMI"] = subset.BMI.clip(0.0, 120.0)

In [32]:
for col in ['EGFR', 'SYSTOLIC_BLOOD_PRESSURE', 'HBA1C', 'SERUM_CHOLESTEROL', 'HIGH_DENSITY_LIPOPROTEIN', 
            'LOW_DENSITY_LIPOPROTEIN', 'TRIGLYCERIDES', 'BMI', 'AGE']:
    print(col, subset[col].min(), subset[col].max())

EGFR 60.0 90.0
SYSTOLIC_BLOOD_PRESSURE 0.0 240.0
HBA1C 0.0 300.0
SERUM_CHOLESTEROL 0.0 20.0
HIGH_DENSITY_LIPOPROTEIN 0.0 30.0
LOW_DENSITY_LIPOPROTEIN 0.0 30.0
TRIGLYCERIDES 0.0 50.0
BMI 0.1 120.0
AGE 25.0 80.0


In [ ]:
# Standardise ranges
for col in ['EGFR', 'SYSTOLIC_BLOOD_PRESSURE', 'HBA1C', 'SERUM_CHOLESTEROL', 'HIGH_DENSITY_LIPOPROTEIN', 
            'LOW_DENSITY_LIPOPROTEIN', 'TRIGLYCERIDES', 'BMI']:
    min_val = round(subset[col].min(), 2)
    max_val = round(subset[col].max(), 2)
    print(col, min_val, max_val)
    subset[col] = subset[col].clip(min_val, max_val)

In [33]:
def improvement(x, lower):
    x = x.values
    if x[0] <= lower and x[1] <= lower:
        return(0.0)
    else:
        val = round(x[0] - x[1], 2)
        return(val)

def bmi_improvement(x):
    x = x.values
    if 18.5 <= x[0] <= 24.9 and 18.5 <= x[1] <= 24.9:
        return (0.0)
    if x[0] < 18.5  or x[1] < 18.5:
        val = round(x[1] - x[0], 2)
        return (val)
    else:
        val = round(x[0] - x[1], 2)
        return(val)

In [34]:
f_s = lambda x: x.rolling(2).apply(lambda x: improvement(x, 120)).shift(-1,fill_value=0) # last visit has 0 rewards
f_a1c = lambda x: x.rolling(2).apply(lambda x: improvement(x, 42)).shift(-1,fill_value=0) # last visit has 0 rewards 
f_bmi = lambda x: x.rolling(2).apply(lambda x: improvement(x, 24.9)).shift(-1,fill_value=0) # last visit has 0 rewards

#data['REWARD_BP'] = data.groupby('PRACTICE_PATIENT_ID').SYSTOLIC_BP.apply(f_s)
#data['REWARD_A1C'] = data.groupby('PRACTICE_PATIENT_ID').HBA1C.apply(f_a1c)
data_sys = subset.groupby('PRACTICE_PATIENT_ID').SYSTOLIC_BLOOD_PRESSURE.apply(f_s)
data_a1c = subset.groupby('PRACTICE_PATIENT_ID').HBA1C.apply(f_a1c)
data_bmi = subset.groupby('PRACTICE_PATIENT_ID').BMI.apply(f_bmi)
#data_n_drugs = data.STATUS.apply(lambda x: -x)

In [35]:
data_a1c_scaled = (2*((data_a1c-min(data_a1c))/(max(data_a1c)-min(data_a1c))))-1
data_sys_scaled = (2*((data_sys-min(data_sys))/(max(data_sys)-min(data_sys))))-1
data_bmi_scaled = (2*((data_bmi-min(data_bmi))/(max(data_bmi)-min(data_bmi))))-1

In [36]:
print("a1c: ", min(data_a1c_scaled), max(data_a1c_scaled))
print("sys: ", min(data_sys_scaled), max(data_sys_scaled))
print("bmi: ", min(data_bmi_scaled), max(data_bmi_scaled))

a1c:  -1.0 1.0
sys:  -1.0 1.0
bmi:  -1.0 1.0


In [37]:
subset['REWARD_SYSTOLIC'] = data_sys_scaled
subset['REWARD_HBA1C'] = data_a1c_scaled
subset['REWARD_BMI'] = data_bmi_scaled
subset['REWARD_A1C_BP'] = data_sys_scaled + data_a1c_scaled
subset['REWARD_ALL'] = data_sys_scaled + data_a1c_scaled + data_bmi_scaled

In [38]:
def discount_cumsum(x, gamma):
    disc_cumsum = np.zeros_like(x)
    disc_cumsum[-1] = x[-1]
    for t in reversed(range(x.shape[0]-1)):
        disc_cumsum[t] = x[t] + gamma * disc_cumsum[t+1]
    return disc_cumsum

In [39]:
subset['RTG_ALL'] = subset['REWARD_ALL'].values
def rtg(group):
    rewards = group['REWARD_ALL'].values
    discounted_rewards = discount_cumsum(rewards, 1.0)
    for i in range(0,len(group)):
        group.iloc[i,group.columns.get_loc('RTG_ALL')]= discounted_rewards[i]
    return group
subset = subset.groupby('PRACTICE_PATIENT_ID').apply(rtg)

In [40]:
subset['RTG_A1C_BP'] = subset['REWARD_A1C_BP'].values
def rtg(group):
    rewards = group['REWARD_A1C_BP'].values
    discounted_rewards = discount_cumsum(rewards, 1.0)
    for i in range(0,len(group)):
        group.iloc[i,group.columns.get_loc('RTG_A1C_BP')]= discounted_rewards[i]
    return group
subset = subset.groupby('PRACTICE_PATIENT_ID').apply(rtg)

In [41]:
subset['RTG_A1C'] = subset['REWARD_HBA1C'].values
def rtg(group):
    rewards = group['REWARD_HBA1C'].values
    discounted_rewards = discount_cumsum(rewards, 1.0)
    for i in range(0,len(group)):
        group.iloc[i,group.columns.get_loc('RTG_A1C')]= discounted_rewards[i]
    return group
subset = subset.groupby('PRACTICE_PATIENT_ID').apply(rtg)

In [42]:
subset['RTG_BMI'] = subset['REWARD_BMI'].values
def rtg(group):
    rewards = group['REWARD_BMI'].values
    discounted_rewards = discount_cumsum(rewards, 1.0)
    for i in range(0,len(group)):
        group.iloc[i,group.columns.get_loc('RTG_BMI')]= discounted_rewards[i]
    return group
subset = subset.groupby('PRACTICE_PATIENT_ID').apply(rtg)

In [43]:
subset['RTG_ALL'] = (2*((subset['RTG_ALL']-min(subset['RTG_ALL']))/(max(subset['RTG_ALL'])-min(subset['RTG_ALL']))))-1
subset['RTG_A1C_BP'] = (2*((subset['RTG_A1C_BP']-min(subset['RTG_A1C_BP']))/(max(subset['RTG_A1C_BP'])-min(subset['RTG_A1C_BP']))))-1
subset['RTG_A1C'] = (2*((subset['RTG_A1C']-min(subset['RTG_A1C']))/(max(subset['RTG_A1C'])-min(subset['RTG_A1C']))))-1
subset['RTG_BMI'] = (2*((subset['RTG_BMI']-min(subset['RTG_BMI']))/(max(subset['RTG_BMI'])-min(subset['RTG_BMI']))))-1

In [44]:
subset['REWARD_SYSTOLIC'] = subset['REWARD_SYSTOLIC'].round(2)
subset['REWARD_HBA1C'] = subset['REWARD_HBA1C'].round(2)
subset['REWARD_BMI'] = subset['REWARD_BMI'].round(2)
subset['REWARD_A1C_BP'] = subset['REWARD_A1C_BP'].round(2)
subset['REWARD_ALL'] = subset['REWARD_ALL'].round(2)
subset['RTG_ALL'] = subset['RTG_ALL'].round(2)
subset['RTG_A1C_BP'] = subset['RTG_A1C_BP'].round(2)
subset['RTG_A1C'] = subset['RTG_A1C'].round(2)
subset['RTG_BMI'] = subset['RTG_BMI'].round(2)

In [45]:
print(np.min(subset['RTG_ALL'].values), np.max(subset['RTG_ALL'].values))
print(np.min(subset['RTG_A1C_BP'].values), np.max(subset['RTG_A1C_BP'].values))
print(np.min(subset['RTG_A1C'].values), np.max(subset['RTG_A1C'].values))
print(np.min(subset['RTG_BMI'].values), np.max(subset['RTG_BMI'].values))

-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0


In [ ]:
subset[["PRACTICE_PATIENT_ID", "REWARD_BMI", "RTG_BMI", "BMI"]].tail(50)

In [46]:
subset.to_csv("processed.csv", index=False)

In [47]:
subset_m = subset[subset['SEX'] == 'M']
subset_f = subset[subset['SEX'] == 'F']

In [48]:
subset_m.to_csv("processed_m.csv", index=False)
subset_f.to_csv("processed_f.csv", index=False)

In [49]:
print(len(subset_m['PRACTICE_PATIENT_ID'].unique()))
print(len(subset_f['PRACTICE_PATIENT_ID'].unique()))

56271
42154


In [55]:
print(56271 - 5627)
print(42154 - 4215)

50644
37939


In [56]:
ids_m = subset_m['PRACTICE_PATIENT_ID'].unique()
ids_f = subset_f['PRACTICE_PATIENT_ID'].unique()

In [57]:
np.random.shuffle(ids_m)
np.random.shuffle(ids_f)
training_m, test_m = ids_m[:50644], ids_m[50644:]
training_f, test_f = ids_f[:37939], ids_f[37939:]

In [58]:
train_df_m = subset_m[subset_m['PRACTICE_PATIENT_ID'].isin(training_m)]
test_df_m = subset_m[subset_m['PRACTICE_PATIENT_ID'].isin(test_m)]

train_df_f = subset_f[subset_f['PRACTICE_PATIENT_ID'].isin(training_f)]
test_df_f = subset_f[subset_f['PRACTICE_PATIENT_ID'].isin(test_f)]

In [59]:
train_df_m.to_csv("../data/train_m.csv", index=False)
test_df_m.to_csv("../data/test_m.csv", index=False)

train_df_f.to_csv("../data/train_f.csv", index=False)
test_df_f.to_csv("../data/test_f.csv", index=False)

In [60]:
np.max(subset['ACTION'])

216

In [32]:
train_df_m.columns

Index(['PRACTICE_PATIENT_ID', 'SEX', 'AGE', 'EGFR', 'SYSTOLIC_BLOOD_PRESSURE',
       'HBA1C', 'SERUM_CHOLESTEROL', 'HIGH_DENSITY_LIPOPROTEIN',
       'LOW_DENSITY_LIPOPROTEIN', 'TRIGLYCERIDES', 'BMI', 'INSULIN',
       'THIAZOLIDINEDIONE', 'SULPHONYLUREAS', 'MEGLITINIDES', 'METFORMIN',
       'GLP1', 'ACARBOSE', 'DPP4I', 'SGLT2', 'ACTION_DESC', 'ACTION',
       'MEDICATION', 'REWARD_SYSTOLIC', 'REWARD_HBA1C', 'REWARD_BMI',
       'REWARD_A1C_BP', 'REWARD_ALL', 'RTG_ALL', 'RTG_A1C_BP', 'RTG_A1C',
       'RTG_BMI'],
      dtype='object')

In [ ]:
print(subset['AGE'].min(), subset['AGE'].max())

In [ ]:
subset['ACTION_DESC'].unique()

In [12]:
data = pd.read_csv("../data/train_f.csv")

In [13]:
df = data.groupby('PRACTICE_PATIENT_ID').size().to_frame().reset_index()

In [14]:
selected = df[(df[0] >= 3) & (df[0] <= 40)]['PRACTICE_PATIENT_ID'].values

In [17]:
data = data[data['PRACTICE_PATIENT_ID'].isin(selected)]

In [18]:
len(data['PRACTICE_PATIENT_ID'].unique())

30348